In [1]:
from pyspark import SparkContext
sc = SparkContext()
from pyspark.sql import SparkSession
# 初始化spark会话
spark = SparkSession.builder.appName('birthrate').getOrCreate()

24/02/22 15:51:05 WARN Utils: Your hostname, dongyanshens-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.64.14.7 instead (on interface en0)
24/02/22 15:51:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/22 15:51:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/02/22 15:51:05 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/02/22 15:51:05 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/02/22 15:51:05 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [3]:
births = spark.read.format("csv") \
        .load('/Users/dongyanshen/Desktop/DYSProjects/TBProject/BirthRate_24F19/BirthRate_24F19/births_train.csv', 
              header=True, inferSchema=True, encoding="utf-8", sep=',')

In [4]:
# 创建转换器、评估器
import pyspark.ml.feature as ft
import pyspark.sql.types as typ

In [5]:
number_cols = []
string_cols = []
for col in births.dtypes:
    
    if col[1] == 'string':
        string_cols.append(col[0])
    else:
        number_cols.append(col[0])


In [6]:
births_typed = births

In [7]:
INT_cols = [f'{col}_INT' for col in string_cols]
stringIndexer = ft.StringIndexer(inputCols=string_cols,outputCols=INT_cols)
model = stringIndexer.fit(births_typed)
births_typed = model.transform(births_typed)

24/02/22 15:51:51 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [8]:
births_typed.show(10)

+----------------------+----------+-----------+-----------+----------------+-----------------+----------------+-------------------+----------------+--------------------+----------+---------+---------+---------+----------------+-----------------+-----------------+----------------------+------------------+------------+-------------+------------+-------------+------------------+-------+----------------------+---------+----------+--------+-----------+----------+------------------------+---------------+-------+--------------+--------+---------------+----------+-------------------------+-------------------+-------------------+------------------------+---------------------+----------------+------------------+---------------+-----------------------+----------------+-----------------------+---------------------+--------------------+-------------------------------------+--------------------------------------+----------------+--------------------------+----------------+-----------------+---------

In [9]:
births_coded = births_typed

In [10]:
OHT_cols = [f'{col}_OHT' for col in string_cols]

# birth place使用one-hot编码
encoder = ft.OneHotEncoder(inputCols=INT_cols,
                           outputCols=OHT_cols)

model = encoder.fit(births_coded)
births_coded = model.transform(births_coded)

In [11]:
births_coded.show(10)

+----------------------+----------+-----------+-----------+----------------+-----------------+----------------+-------------------+----------------+--------------------+----------+---------+---------+---------+----------------+-----------------+-----------------+----------------------+------------------+------------+-------------+------------+-------------+------------------+-------+----------------------+---------+----------+--------+-----------+----------+------------------------+---------------+-------+--------------+--------+---------------+----------+-------------------------+-------------------+-------------------+------------------------+---------------------+----------------+------------------+---------------+-----------------------+----------------+-----------------------+---------------------+--------------------+-------------------------------------+--------------------------------------+----------------+--------------------------+----------------+-----------------+---------

In [12]:
assembler = ft.VectorAssembler(inputCols=number_cols+OHT_cols[1:], outputCol="features") 
births_coded = assembler.transform(births_coded)

In [13]:
# 创建一个评估器
import pyspark.ml.classification as cl

logistic = cl.LogisticRegression(maxIter=10,
                                regParam=0.01,
                                featuresCol='features',
                                labelCol='INFANT_ALIVE_AT_REPORT_INT')

In [14]:
train_data, val_data, test_data = births_coded.randomSplit([0.5, 0.2, 0.3], seed=42)

In [15]:
model = logistic.fit(train_data)

24/02/22 15:52:11 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


In [17]:
test_model = model.transform(test_data)

In [18]:
# 评估模型性能
import pyspark.ml.evaluation as ev

evaluator = ev.BinaryClassificationEvaluator(
    rawPredictionCol='probability',
    labelCol='INFANT_ALIVE_AT_REPORT_INT'
)

print(evaluator.evaluate(test_model, {evaluator.metricName:'areaUnderROC'}))
print(evaluator.evaluate(test_model, {evaluator.metricName:'areaUnderPR'}))

0.9729032630133321
CodeCache: size=131072Kb used=45026Kb max_used=45102Kb free=86045Kb
 bounds [0x000000010a1e0000, 0x000000010ce30000, 0x00000001121e0000]
 total_blobs=15545 nmethods=14345 adapters=1111
 compilation: disabled (not enough contiguous free space left)


Java HotSpot(TM) 64-Bit Server VM warning: CodeCache is full. Compiler has been disabled.
Java HotSpot(TM) 64-Bit Server VM warning: Try increasing the code cache size using -XX:ReservedCodeCacheSize=


0.9796652205149617


In [20]:
# 评估模型性能
import pyspark.ml.evaluation as ev

evaluator = ev.BinaryClassificationEvaluator(
    rawPredictionCol='probability',
    labelCol='INFANT_ALIVE_AT_REPORT_INT'
)

print(evaluator.evaluate(test_model, {evaluator.metricName:'areaUnderROC'}))
print(evaluator.evaluate(test_model, {evaluator.metricName:'areaUnderPR'}))

0.9744146485263353
0.9795736779565234
